# Text Rank from Scratch

In [38]:
import numpy as np
import pandas as pd
import nltk
import re
import contractions
nltk.download('punkt') # one time execution

[nltk_data] Downloading package punkt to /Users/ralfweber/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [213]:
%autosave 120
pd.set_option('display.max_colwidth',1000)

Autosaving every 120 seconds


# Load Glove Embeddings

In [214]:
import numpy as np

def load_glove_vectors(fn):
    print("Loading GloVe Model...")
    with open( fn,'r', encoding='utf8') as glove_vector_file:
        model = {}
        for line in glove_vector_file:
            parts = line.split()
            word = parts[0]
            embedding = np.array([float(val) for val in parts[1:]])
            model[word] = embedding
        print("Loaded {} words in GloVe model".format(len(model)))
    return model

glove_vectors = load_glove_vectors('data/glove.6B.50d.txt') # download this from: https://www.kaggle.com/datasets/watts2/glove6b50dtxt

Loading GloVe Model...
Loaded 400000 words in GloVe model


# Read Articles

In [272]:
articles = pd.read_csv("data/news-article-categories.csv")

In [273]:
# Convert empty strings to NaN
articles.replace('', np.nan, inplace=True)

# Drop rows with NaN values
articles.dropna(inplace=True)

articles.head(10)

,category,title,text
0,ARTS & CULTURE,"Modeling Agencies Enabled Sexual Predators For Years, Former Agent Says","In October 2017, Carolyn Kramer received a disturbing phone call. The former modeling agent listened intently as a model she used to represent told her that a famous French photographer, who still shoots for top publications, raped her when she was 16. Shortly after meeting the man at a restaurant in 1983, the model said she blacked out after drinking one glass of champagne, then woke up in his bed the next morning with a sore and bruised vagina. The woman, who preferred to remain anonymous, confirmed this story with HuffPost, but did not want to name the photographer for fear of legal repercussions. “He was one of the photographers that agents and clients and young girls basically knew was lecherous,” Kramer said, claiming that she and other agents sent their models to him in the '80s anyway. “[But] what I didn't realize [at the time] is that he was raping girls.” Kramer said she broke down crying after the woman relayed her story of assault. “In that moment I felt that I let dow..."
1,ARTS & CULTURE,Actor Jeff Hiller Talks 'Bright Colors And Bold Patterns' and More (AUDIO),"This week I talked with actor Jeff Hiller about the hit Off Broadway play Bright Colors And Bold Patterns that he'll be joining on January 17th with a new opening night scheduled for February 4th. Hiller (Nightcap, 30 Rock & Broadway's Bloody Bloody Andrew Jackson) will step into the starring role of this devastatingly funny hit play directed by Michael Urie (Torch Song, Buyer & Cellar, Ugly Betty) and written by Drew Droege the play's original star. Hiller will continue the play's triumphant Off Broadway run through February 25th at the SoHo Playhouse in NYC. Bright Colors And Bold Patterns is about Josh and Brennan who are about to get married in Palm Springs on a lovely Saturday afternoon. However, the night before becomes a drunken, drug-fueled riot because their friend Gerry arrives furious that their invitation says: “Please refrain from wearing bright colors or bold patterns.” The play is produced by Zach Laks in association with Riki Kane Larimer and features set design by ..."
2,ARTS & CULTURE,New Yorker Cover Puts Trump 'In The Hole' After 'Racist' Comment,"The New Yorker is taking on President Donald Trump after he asked why the U.S. would welcome immigrants from “shithole” places like Haiti and African countries during a bipartisan Oval Office meeting on Thursday. “In the Hole,” by artist Anthony Russo, responds to the president's comment, which has been decried as racist by the United Nations, with a stark illustration for the magazine's cover in its Jan. 22 issue. On a field of white, the president's yellowish sweep of hair is just visible from the depths of a black hole: On Thursday, Trump sparked widespread criticism after he reportedly questioned why the U.S. should restore protections for immigrants from Haiti and parts of Africa. “Why are we having all these people from shithole countries come here?” the president said, sources told The Washington Post. “We should have more people from places like Norway,” he added. The New York Times, NBC News, BuzzFeed and CNN confirmed reports of the comment. Sen. Dick Durbin (D-Ill.) wen..."
3,ARTS & CULTURE,Man Surprises Girlfriend By Drawing Them In Different Animation Styles,"Kellen Hickey, a 26-year-old who lives in Hudson, Wisconsin, has gift giving down to a fine art. He drew himself and his girlfriend Lindsay Brinkman, 24, in 10 different animation styles and gave the illustrations to her on Christmas. Hickey told HuffPost that he got the idea a few months ago when he decided to draw the couple as “Futurama” and “Adventure Time” characters and Brinkman absolutely loved them. Her enthusiasm inspired him to draw the two of them in the style of all the cartoons they had bonded over since they began dating in high school. “I am fortunate enough to have the ability to create something t

In [274]:
nltk.download('stopwords')
from nltk.corpus import stopwords # in case of SSL error, see here: https://stackoverflow.com/a/42890688
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ralfweber/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [275]:
from nltk.tokenize import sent_tokenize, word_tokenize

CLEAN_PATTERN = r'[^a-zA-z\s]'

def clean(word):
    return re.sub(CLEAN_PATTERN, '', word)

def clean_sentence(sentence):
    sentence = [clean(word) for word in sentence]
    return [word for word in sentence if word]

def clean_sentences(sentences):
    return [clean_sentence(sentence) for sentence in sentences]

def make_lowercase(sentence):
    return [word.lower() for word in sentence]

def remove_stopwords(sentence):
    words = [word for word in sentence if word not in stop_words]
    return [word for word in words if len(word) > 0]

def tokenize_words(sentences):
    return [word_tokenize(sentence) 
              for sentence in sentences]

def fix_contractions(sentences):
    return [contractions.fix(sentence) for sentence in sentences]

articles['SentencesInArticle'] = articles.text.apply(sent_tokenize)
articles['WordsInSentences'] = articles.SentencesInArticle \
              .apply(fix_contractions)\
              .apply(make_lowercase)\
              .apply(tokenize_words)\
              .apply(remove_stopwords)\
              .apply(clean_sentences)

In [276]:
articles = articles[['SentencesInArticle', 'WordsInSentences']]
articles.head(2)

,SentencesInArticle,WordsInSentences
0,"[In October 2017, Carolyn Kramer received a disturbing phone call., The former modeling agent listened intently as a model she used to represent told her that a famous French photographer, who still shoots for top publications, raped her when she was 16., Shortly after meeting the man at a restaurant in 1983, the model said she blacked out after drinking one glass of champagne, then woke up in his bed the next morning with a sore and bruised vagina., The woman, who preferred to remain anonymous, confirmed this story with HuffPost, but did not want to name the photographer for fear of legal repercussions., “He was one of the photographers that agents and clients and young girls basically knew was lecherous,” Kramer said, claiming that she and other agents sent their models to him in the '80s anyway., “[But] what I didn't realize [at the time] is that he was raping girls.” Kramer said she broke down crying after the woman relayed her story of assault., “In that moment I felt that I ...","[[in, october, carolyn, kramer, received, a, disturbing, phone, call], [the, former, modeling, agent, listened, intently, as, a, model, she, used, to, represent, told, her, that, a, famous, french, photographer, who, still, shoots, for, top, publications, raped, her, when, she, was], [shortly, after, meeting, the, man, at, a, restaurant, in, the, model, said, she, blacked, out, after, drinking, one, glass, of, champagne, then, woke, up, in, his, bed, the, next, morning, with, a, sore, and, bruised, vagina], [the, woman, who, preferred, to, remain, anonymous, confirmed, this, story, with, huffpost, but, did, not, want, to, name, the, photographer, for, fear, of, legal, repercussions], [he, was, one, of, the, photographers, that, agents, and, clients, and, young, girls, basically, knew, was, lecherous, kramer, said, claiming, that, she, and, other, agents, sent, their, models, to, him, in, the, s, anyway], [[, but, ], what, i, did, not, realize, [, at, the, time, ], is, that, he, was..."
1,"[This week I talked with actor Jeff Hiller about the hit Off Broadway play Bright Colors And Bold Patterns that he'll be joining on January 17th with a new opening night scheduled for February 4th., Hiller (Nightcap, 30 Rock & Broadway's Bloody Bloody Andrew Jackson) will step into the starring role of this devastatingly funny hit play directed by Michael Urie (Torch Song, Buyer & Cellar, Ugly Betty) and written by Drew Droege the play's original star., Hiller will continue the play's triumphant Off Broadway run through February 25th at the SoHo Playhouse in NYC., Bright Colors And Bold Patterns is about Josh and Brennan who are about to get married in Palm Springs on a lovely Saturday afternoon., However, the night before becomes a drunken, drug-fueled riot because their friend Gerry arrives furious that their invitation says: “Please refrain from wearing bright colors or bold patterns.” The play is produced by Zach Laks in association with Riki Kane Larimer and features set desig...","[[this, week, i, talked, with, actor, jeff, hiller, about, the, hit, off, broadway, play, bright, colors, and, bold, patterns, that, he, will, be, joining, on, january, th, with, a, new, opening, night, scheduled, for, february, th], [hiller, nightcap, rock, broadway, s, bloody, bloody, andrew, jackson, will, step, into, the, starring, role, of, this, devastatingly, funny, hit, play, directed, by, michael, urie, torch, song, buyer, cellar, ugly, betty, and, written, by, drew, droege, the, play, s, original, star], [hiller, will, continue, the, play, s, triumphant, off, broadway, run, through, february, th, at, the, soho, playhouse, in, nyc], [bright, colors, and, bold, patterns, is, about, josh, and, brennan, who, are, about, to, get, married, in, palm, springs, on, a, lovely, saturday, afternoon], [however, the, night, before, becomes, a, drunken, drugfueled, riot, because, their, friend, gerry, arrives, furious, that, their, invitation, 

## Vectorized Sentences

In [277]:
articles.head(2)

,SentencesInArticle,WordsInSentences
0,"[In October 2017, Carolyn Kramer received a disturbing phone call., The former modeling agent listened intently as a model she used to represent told her that a famous French photographer, who still shoots for top publications, raped her when she was 16., Shortly after meeting the man at a restaurant in 1983, the model said she blacked out after drinking one glass of champagne, then woke up in his bed the next morning with a sore and bruised vagina., The woman, who preferred to remain anonymous, confirmed this story with HuffPost, but did not want to name the photographer for fear of legal repercussions., “He was one of the photographers that agents and clients and young girls basically knew was lecherous,” Kramer said, claiming that she and other agents sent their models to him in the '80s anyway., “[But] what I didn't realize [at the time] is that he was raping girls.” Kramer said she broke down crying after the woman relayed her story of assault., “In that moment I felt that I ...","[[in, october, carolyn, kramer, received, a, disturbing, phone, call], [the, former, modeling, agent, listened, intently, as, a, model, she, used, to, represent, told, her, that, a, famous, french, photographer, who, still, shoots, for, top, publications, raped, her, when, she, was], [shortly, after, meeting, the, man, at, a, restaurant, in, the, model, said, she, blacked, out, after, drinking, one, glass, of, champagne, then, woke, up, in, his, bed, the, next, morning, with, a, sore, and, bruised, vagina], [the, woman, who, preferred, to, remain, anonymous, confirmed, this, story, with, huffpost, but, did, not, want, to, name, the, photographer, for, fear, of, legal, repercussions], [he, was, one, of, the, photographers, that, agents, and, clients, and, young, girls, basically, knew, was, lecherous, kramer, said, claiming, that, she, and, other, agents, sent, their, models, to, him, in, the, s, anyway], [[, but, ], what, i, did, not, realize, [, at, the, time, ], is, that, he, was..."
1,"[This week I talked with actor Jeff Hiller about the hit Off Broadway play Bright Colors And Bold Patterns that he'll be joining on January 17th with a new opening night scheduled for February 4th., Hiller (Nightcap, 30 Rock & Broadway's Bloody Bloody Andrew Jackson) will step into the starring role of this devastatingly funny hit play directed by Michael Urie (Torch Song, Buyer & Cellar, Ugly Betty) and written by Drew Droege the play's original star., Hiller will continue the play's triumphant Off Broadway run through February 25th at the SoHo Playhouse in NYC., Bright Colors And Bold Patterns is about Josh and Brennan who are about to get married in Palm Springs on a lovely Saturday afternoon., However, the night before becomes a drunken, drug-fueled riot because their friend Gerry arrives furious that their invitation says: “Please refrain from wearing bright colors or bold patterns.” The play is produced by Zach Laks in association with Riki Kane Larimer and features set desig...","[[this, week, i, talked, with, actor, jeff, hiller, about, the, hit, off, broadway, play, bright, colors, and, bold, patterns, that, he, will, be, joining, on, january, th, with, a, new, opening, night, scheduled, for, february, th], [hiller, nightcap, rock, broadway, s, bloody, bloody, andrew, jackson, will, step, into, the, starring, role, of, this, devastatingly, funny, hit, play, directed, by, michael, urie, torch, song, buyer, cellar, ugly, betty, and, written, by, drew, droege, the, play, s, original, star], [hiller, will, continue, the, play, s, triumphant, off, broadway, run, through, february, th, at, the, soho, playhouse, in, nyc], [bright, colors, and, bold, patterns, is, about, josh, and, brennan, who, are, about, to, get, married, in, palm, springs, on, a, lovely, saturday, afternoon], [however, the, night, before, becomes, a, drunken, drugfueled, riot, because, their, friend, gerry, arrives, furious, that, their, invitation, 

In [278]:
VECTOR_SIZE = 50
EMPTY_VECTOR = np.zeros(VECTOR_SIZE)

def sentence_vector(sentence):
    return sum([glove_vectors.get(word, EMPTY_VECTOR) 
                  for word in sentence]) / (len(sentence) if len(sentence) != 0 else 1)

def sentences_to_vectors(sentences):
    return [sentence_vector(sentence) 
              for sentence in sentences]

In [279]:
articles['SentenceVector'] = articles.WordsInSentences.apply(sentences_to_vectors)

## Similarity Matrix

In [280]:
from sklearn.metrics.pairwise import cosine_similarity

def create_similarity_matrix(sentence_vectors):
    sim_mat = np.zeros([len(sentence_vectors), len(sentence_vectors)])
    for i in range(len(sentence_vectors)):
        for j in range(len(sentence_vectors)):
            element_i = sentence_vectors[i].reshape(1,VECTOR_SIZE)
            element_j = sentence_vectors[j].reshape(1,VECTOR_SIZE)
            sim_mat[i][j] = cosine_similarity(element_i,
                                              element_j)[0,0]
    return sim_mat

In [281]:
articles['SimMatrix'] = \
     articles.SentenceVector.apply(create_similarity_matrix)

In [282]:
import networkx as nx

def compute_graph(sim_matrix):
    nx_graph = nx.from_numpy_array(sim_matrix)
    scores = nx.pagerank(nx_graph, max_iter=500)
    return scores

## Graph

In [283]:
articles['Graph'] = articles.SimMatrix.apply(compute_graph)

In [284]:
articles.head(3)

,SentencesInArticle,WordsInSentences,SentenceVector,SimMatrix,Graph
0,"[In October 2017, Carolyn Kramer received a disturbing phone call., The former modeling agent listened intently as a model she used to represent told her that a famous French photographer, who still shoots for top publications, raped her when she was 16., Shortly after meeting the man at a restaurant in 1983, the model said she blacked out after drinking one glass of champagne, then woke up in his bed the next morning with a sore and bruised vagina., The woman, who preferred to remain anonymous, confirmed this story with HuffPost, but did not want to name the photographer for fear of legal repercussions., “He was one of the photographers that agents and clients and young girls basically knew was lecherous,” Kramer said, claiming that she and other agents sent their models to him in the '80s anyway., “[But] what I didn't realize [at the time] is that he was raping girls.” Kramer said she broke down crying after the woman relayed her story of assault., “In that moment I felt that I ...","[[in, october, carolyn, kramer, received, a, disturbing, phone, call], [the, former, modeling, agent, listened, intently, as, a, model, she, used, to, represent, told, her, that, a, famous, french, photographer, who, still, shoots, for, top, publications, raped, her, when, she, was], [shortly, after, meeting, the, man, at, a, restaurant, in, the, model, said, she, blacked, out, after, drinking, one, glass, of, champagne, then, woke, up, in, his, bed, the, next, morning, with, a, sore, and, bruised, vagina], [the, woman, who, preferred, to, remain, anonymous, confirmed, this, story, with, huffpost, but, did, not, want, to, name, the, photographer, for, fear, of, legal, repercussions], [he, was, one, of, the, photographers, that, agents, and, clients, and, young, girls, basically, knew, was, lecherous, kramer, said, claiming, that, she, and, other, agents, sent, their, models, to, him, in, the, s, anyway], [[, but, ], what, i, did, not, realize, [, at, the, time, ], is, that, he, was...","[[0.12994258888888888, 0.266952, 0.057090877777777785, 0.16973233333333335, 0.24598523333333336, 0.21671600000000002, -0.5938655555555555, -0.11241122222222222, -0.08604877777777778, -0.13432900000000003, -0.2299411111111111, -0.06699044444444444, -0.12497687777777777, -0.208422, 0.6825866666666667, -0.12708311111111112, -0.5678577777777777, -0.2223727777777778, -0.3412744444444443, -0.05024255555555554, 0.31370433333333336, 0.32598333333333335, 0.19989677777777784, 0.0006989999999999805, 0.22038888888888888, -1.3321777777777777, -0.07470222222222221, -0.009619888888888897, -0.03724033333333334, -0.12524988888888888, 2.283092222222222, -0.19608666666666666, -0.16698711111111111, -0.45470555555555553, -0.03763433333333334, -0.23824755555555555, 0.22576733333333332, -0.081817, 0.16799266666666662, -0.19468892222222225, 0.13659666666666667, 0.4050569777777778, -0.1364191111111111, -0.13753902222222222, -0.02407033333333331, 0.04287244444444445, -0.04981133333333333, -0.045210222222222...","[[1.0, 0.9058512560063415, 0.8843496707451783, 0.9044396825560004, 0.8988321718054801, 0.9118362546085397, 0.8798485791769182, 0.8561647610416898, 0.8922867978408691, 0.9195244345062042, 0.8647582822942053, 0.9019217999865002, 0.8821819138830912, 0.8883468214736762, 0.903446173655496, 0.8610538948619471, 0.8323685255419793, 0.9184692075335579, 0.8646186964747499, 0.8617520812761341, 0.9035784175362052, 0.8879786632272613, 0.8790919472899136, 0.8923130844685031, 0.8917879919187983, 0.9043410005664844, 0.8909261284632819, 0.9198803445992786, 0.8127941370621818, 0.8798526056096261, 0.9211790143215365, 0.8475033768570281, 0.8814882556708566, 0.8917585223710943, 0.8433023611042434, 0.8806532905083848, 0.8977804624817841, 0.8800492567350834, 0.7897873521349946, 0.887256249481563, 0.8080899515979707, 0.8957596762721823, 0.8834975240559813, 0.8780143242478503, 0.8822785256674515, 0.8987170221126601, 0.8986077043151316,

## Ranked Sentences

In [285]:
def get_ranked_sentences(sentences, scores, n=3):
    top_scores = sorted(((scores[i],s) 
                         for i,s in enumerate(sentences)), 
                                reverse=True)
    top_n_sentences = [sentence 
                        for score,sentence in top_scores[:n]]
    return " ".join(top_n_sentences)

articles['Summary'] = articles.apply(lambda d: 
                                     get_ranked_sentences(d.SentencesInArticle, 
                                                          d.Graph), axis=1)

## Summary

In [286]:
articles.loc[0].Summary

"“To say there was an arrangement of financial exchange and sexual favors would be completely out of whack,” he said, adding, however, that “if you're looking after a whole bunch of good-looking ladies who are running around Manhattan, you're going to be running into trouble here and there sometimes.”)  But at 24, Kramer found herself employed by the modeling world's gold standard, an agency that represented Cindy Crawford and Linda Evangelista, where she didn't expect to encounter abusive men. I think we would try not to work with a lot of those photographers if we heard problems.” Trudi Tapscott, who worked as an agent and director at Elite from the early '80s until the early '90s, reiterated Gotlib's point, explaining that she told young women about potentially creepy photographers ahead of time. But then I thought I was helping them survive, which sounds so stupid now.” As for alleged in-house predators such as Marie, Gotlib said, “I knew nothing about anything that might have been

In [287]:
articles.loc[1].Summary

"This week I talked with actor Jeff Hiller about the hit Off Broadway play Bright Colors And Bold Patterns that he'll be joining on January 17th with a new opening night scheduled for February 4th. LISTEN:   When asked how he sees our LGBTQ community moving forward in this Trump administration Hiller stated: Jeff Hiller is a regular performer at the UCB Theatre in both New York City and Los Angeles. However, the night before becomes a drunken, drug-fueled riot because their friend Gerry arrives furious that their invitation says: “Please refrain from wearing bright colors or bold patterns.” The play is produced by Zach Laks in association with Riki Kane Larimer and features set design by Dara Wishingrad."